## Turn your S&P500 portfolio into synthetic in-the-money calls.

#### This notebook researches a strategy in which:
1. You buy the S&P 500 (using an ETF);
2. You choose the value ```put_perc_otm```, which is the **percent below the S&P purchase price** to use as the strike price of a put that limits your downside exposure;
3. You choose the value ```years_to_hedge```, which is the duration of the put;
4. Actions to take as the S&P price moves and time passes:
  * S&P rises to  ``` 2 * put_perc_otm``` above current hedge strike: 
    * you sell the previous put, and purhase another put at a **higher** strike, and for the full ```years_to_hedge```, effectively buying diagonal put spreads
  * S&P falls to ```2 * put_perc_otm``` below the current hedge strike: 
    * you sell the previous put, and purhase another put at a **lower** strike, and for the full ```years_to_hedge```, effectively selling diagonal put spreads

#### The main benefit of this strategy
* The strategy is designed to provide continual insurance  of your long S&P position, using a rolling series of puts.  These puts effectively turn your S&P position into a call that still collects dividends.
* Depending on where the price of the S&P 500 is relative to the current put strike, you will either have a position that is long an in-the-money call (as the S&P 500 rises to newer all time highs, or an out of the money all (as the S&P falls from those all time highs).


## IF YOU WANT TO SEE WARNINGS, COMMENT THIS OUT

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from IPython.core.display import  HTML
from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
from dateutil.relativedelta import relativedelta
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr
from scipy.stats import norm

from ipysheet import from_dataframe,to_dataframe
# importlib.reload(build_db)

### important global variables

#### Step 01: define important parameters that are used in successive steps below

In [3]:
# define how far out of the money you want your hedge
# put_perc_otm = 0.125
put_perc_otm = 0.14
years_to_hedge = 2
# set the target percentage of stock
rebal_target = .6
# set the percentage of stock in the portfolio that forces a rebalance
rebal_adjust = .7


Define and display important dates and values

In [4]:
sp_data_end_date = datetime.datetime.now()
sp_data_beg_date = sp_data_end_date - relativedelta(years=30)
beg_date_str = datetime.datetime.strftime(sp_data_beg_date,'%Y-%m-%d')
end_date_str = datetime.datetime.strftime(sp_data_end_date,'%Y-%m-%d')
print(f"using put hedge {put_perc_otm * 100} percent out of the money")
print(f"put hedge duration = {years_to_hedge} years (this can be a fraction like 1.5)")
print(f"S&P history starts on {beg_date_str} and ends on {end_date_str}")


using put hedge 14.000000000000002 percent out of the money
put hedge duration = 2 years (this can be a fraction like 1.5)
S&P history starts on 1990-05-18 and ends on 2020-05-18


#### Step 02: define important functions that are used below

In [5]:
def str_to_yyyymmdd(d,sep='-'):
    try:
        dt = datetime.datetime.strptime(str(d)[:10],f'%Y{sep}%m{sep}%d')
    except:
        return None
    s = '%04d%02d%02d' %(dt.year,dt.month,dt.day)
    return int(s)

def str_to_date(d,sep='-'):
    try:
        dt = datetime.datetime.strptime(str(d)[:10],f'%Y{sep}%m{sep}%d')
    except:
        return None
    return dt


def fetch_history(symbol,dt_beg,dt_end):
    df = pdr.DataReader(symbol, 'yahoo', dt_beg, dt_end)
    # move index to date column, sort and recreate index
    df['date'] = df.index
    df = df.sort_values('date')
    df.index = list(range(len(df)))
    # make adj close the close
    df = df.drop(['Adj Close'],axis=1)
    cols = df.columns.values 
    cols_dict = {c:c[0].lower() + c[1:] for c in cols}
    df = df.rename(columns = cols_dict)
    df['settle_date'] = df.date.apply(str_to_yyyymmdd)
    return df


In [6]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,width=800,height=400,
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        autosize=False,
        width=width,
        height=height,
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    return fig

def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

## Create a hedge strategy and use data on ^GSPC from yahoo 

#### Step 03: get data

In [7]:
# dt_end = datetime.datetime.now()
# dt_beg = dt_end - datetime.timedelta(365*30)
df_spy = fetch_history('^GSPC', sp_data_beg_date, sp_data_end_date)
df_vix = fetch_history('^VIX',sp_data_beg_date,sp_data_end_date)
df_tnx = fetch_history('^TNX',sp_data_beg_date,sp_data_end_date)
df_1yr_rate = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DGS1&scale=left&cosd=1962-01-02&coed=2020-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2009-06-01&line_index=1&transformation=lin&vintage_date=2020-05-04&revision_date=2020-05-04&nd=1962-01-02')
dates_1yr = [datetime.datetime.strptime(d,'%Y-%m-%d') for d in df_1yr_rate.DATE.values]
df_1yr_rate['settle_date'] = [int(d.year)*100*100+int(d.month)*100+int(d.day) for d in dates_1yr]
df_1yr_rate = df_1yr_rate.rename(columns={'DGS1':'rate'})
df_1yr_rate = df_1yr_rate[['settle_date','rate']]
df_1yr_rate.rate = [0.0 if s=='.' else float(s) / 100 for s in df_1yr_rate.rate]
df_div = pd.read_csv('sp_div_yield.csv')

### Calculate the cost/revenue of the hedge.
The put hedge that you will buy will initially be below the current SP price by a percentage which you set in the variable ```put_perc_otm```.  When the price of the SP rises high enough so that you can raise the strike price of the hedge, you sell the current put (if there is any value in it) and buy a new put that is ```put_perc_otm``` percent higher than the previous put.  In this way, you are not letting your hedge get too far from the money.


* Remember that, since you are comparing this put strategy to "Buy-And-Hold"
  * Rolls to a higher strike are a cost to the strategy
  * Rolls to a lower strike are revenue to the strategy.

## This is the main step.
#### Step 04: create the dataframe called ```dft``` which has all of the strategy info, incluing hedge values.



In [8]:
# create a lambda that converts yyyymmdd integer to a datetime object
yyyymmdd_to_dt = lambda v:datetime.datetime(
        int(str(v)[0:4]),int(str(v)[4:6]),int(str(v)[6:8])
)

# grab only the relevant columns from df_spy
dft = df_spy[['settle_date','close','high','low']]
# create a datetime settle date, along with the yyyymmdd settle_date column
dft['settle_dt'] = dft.settle_date.apply(yyyymmdd_to_dt)
# initialize currrent_strike, which is below the money
current_long_price = dft.iloc[0].close
current_strike = current_long_price * (1 - put_perc_otm)
current_strike_array = [current_strike]

# create an array of high and low values, to speed up loop processing    
m = dft[['high','low']].values

# Main loop is here, which determines the hedge dates, and the value of the put
#   options used in each hedge
for i in tqdm_notebook(range(1,len(m))):
    # get high and low
    curr_high = m[i][0]
    curr_low = m[i][1]
    # if the price rises past current_strike * (1 + put_perc_otm) * (1+ put_perc_otm)
    #   then you want to roll the put strike up, buy essentially BUYING a put spread
    if curr_high  > current_strike * (1 + put_perc_otm)**2:
        # roll strikes up, like buying put spreads as market goes up
#         current_strike = current_strike * (1 + put_perc_otm)**2
        current_strike = current_strike * (1 + put_perc_otm)
    # if the price falls below current_strike * (1 - put_perc_otm) * (1- put_perc_otm)
    #   then you want to roll the put strike down, buy essentially SELLING a put spread
    elif curr_low < current_strike * (1 - put_perc_otm)**2:
        # roll strikes down (like selling put spreads as market drops)
#         current_strike = current_strike * (1 - put_perc_otm)**2
        current_strike = current_strike * (1 - put_perc_otm)
    current_strike_array.append(current_strike)
    
# update dft with the current_strike array    
dft['current_hedge_strike'] = current_strike_array
# also add in the previous strike, so that you can tell when you have to buy or sell
#  put spreads
dft['prev_hedge_strike'] = dft.current_hedge_strike.shift(1)
#  The next 2 lines is where you determine the dates on which you execute hedges
dft.loc[dft.prev_hedge_strike!=dft.current_hedge_strike,'time_to_hedge'] = True
dft.loc[dft.prev_hedge_strike==dft.current_hedge_strike,'time_to_hedge'] = False

#
dft.loc[dft.time_to_hedge,'hedge_date'] = dft.loc[dft.time_to_hedge].settle_date
dft.loc[dft.time_to_hedge==False,'hedge_date'] = dft.settle_date.min()
dft.hedge_date = dft.hedge_date.expanding(min_periods=1).max()
dft.hedge_date = dft.hedge_date.apply(yyyymmdd_to_dt)
dft['prev_hedge_date'] = dft.hedge_date.shift(1)
dft['days_of_hedge'] = (dft.settle_dt - dft.hedge_date).dt.days
dft.loc[dft.time_to_hedge,'days_of_hedge'] = (dft[dft.time_to_hedge].hedge_date - dft[dft.time_to_hedge].prev_hedge_date).dt.days
df_vix2 = df_vix[['settle_date','close']]
df_vix2 = df_vix2.rename(columns={'close':'atm_vol'})
df_vix2.atm_vol = df_vix2.atm_vol / 100
dft = dft.merge(df_vix2,on='settle_date',how='inner')

# add in rate
dft = dft.merge(df_1yr_rate,on='settle_date',how='inner')

# add div yield
dft['year'] = dft.settle_date.apply(lambda v:int(str(v)[0:4]))
dft = dft.merge(df_div,on='year',how='inner')

# Now calculate cost/revenue of buying puts
def _calc_put_spread(r):
    '''
    !! This should only be exexuted on rows of dft where dft.time_to_hedge==True !!
    
    Calculate the value of the option spread where the legs are: 
      1. the current_hedge_strike 
      2. previous hedge strike
    The value will be positive if you are buying the spread b/c you are rolling
      the previous hedge forward (to a higher strike).
    The value will be negative if you are selling the spread b/c you are rolling
      the previous hedge backward (to a lower strike)
    '''
     #black.black(flag, F, K, t, r, sigma)
    atm_vol = r.atm_vol
    if r.prev_hedge_strike < r.current_hedge_strike: 
        curr_strike_vol = atm_vol + .04 
        prev_strike_vol = atm_vol + .08
    else:
        curr_strike_vol = atm_vol - .04 
        prev_strike_vol = atm_vol - .06
    
    days_left_in_prev_hedge = (r.hedge_date - r.prev_hedge_date).days

    # calculate remaining of previous hedge
    if r.prev_hedge_strike < r.current_hedge_strike:
        # we are rolling up b/c the market is put_perc_otm ABOVE the current_hedge
        underlying_price = r.current_hedge_strike * (1+put_perc_otm)
#         curr_hedge =  black.black('p', underlying_price, r.current_hedge_strike, years_to_hedge, .02, curr_strike_vol)
        curr_hedge =  black.black('p', underlying_price, r.current_hedge_strike, years_to_hedge,r.rate, curr_strike_vol)
        if days_left_in_prev_hedge > years_to_hedge*365:
            remaining_opt_value = 0
        else:
            time_remaining = days_left_in_prev_hedge/(years_to_hedge*365)
#             remaining_opt_value = black.black('p', underlying_price, r.prev_hedge_strike, 
#                                               time_remaining, .02, prev_strike_vol)
            remaining_opt_value = black.black('p', underlying_price, r.prev_hedge_strike, 
                                              time_remaining, r.rate, prev_strike_vol)
    else:
        # we are rolling down b/c the market is put_perc_otm BELOW the current_hedge
        underlying_price = r.current_hedge_strike * (1-put_perc_otm)
#         curr_hedge =  black.black('p', underlying_price, r.current_hedge_strike, years_to_hedge, .02, curr_strike_vol)
        curr_hedge =  black.black('p', underlying_price, r.current_hedge_strike, years_to_hedge, r.rate, curr_strike_vol)
        if days_left_in_prev_hedge > years_to_hedge*365:
            remaining_opt_value = r.prev_hedge_strike - underlying_price
        else:
#             remaining_opt_value =  black.black('p', underlying_price, r.prev_hedge_strike, years_to_hedge, .02, prev_strike_vol)
            remaining_opt_value =  black.black('p', underlying_price, r.prev_hedge_strike, years_to_hedge, r.rate, prev_strike_vol)
        
   
    return curr_hedge - remaining_opt_value

dft.loc[dft.time_to_hedge,'hedge'] = dft.loc[dft.time_to_hedge].apply(_calc_put_spread,axis=1)
dft.loc[dft.time_to_hedge==False,'hedge'] = 0
dft['hedge_cumulative'] = [0] + dft.iloc[1:].hedge.cumsum().values.tolist()

#### Step 05: graph the close of ^GSCP (SP 500) vs the strikes at which we hedged

In [9]:
dft_to_plot = dft[['settle_date','close','current_hedge_strike']]
# dft_to_plot = dft_to_plot[(dft_to_plot.settle_date>20080701) & ()]

iplot(plotly_plot(df_in=dft_to_plot,x_column='settle_date'))

#### Step 06: list all put purchases when rolling to a higher strike as the market rises.
You are essentially buying a put spread on every roll higher.

In [10]:
dft[(dft.time_to_hedge) & (dft.prev_hedge_strike<dft.current_hedge_strike)]

,settle_date,close,high,low,settle_dt,current_hedge_strike,prev_hedge_strike,time_to_hedge,hedge_date,prev_hedge_date,days_of_hedge,atm_vol,rate,year,div_yield,hedge,hedge_cumulative
404,19911224,399.329987,401.790009,396.820007,1991-12-24,350.983200,307.880000,True,1991-12-24,1990-05-21,582.0,0.1567,0.0418,1991,0.0311,16.048760,16.048760
709,19930310,456.329987,456.339996,452.700012,1993-03-10,400.120848,350.983200,True,1993-03-10,1991-12-24,442.0,0.1391,0.0340,1993,0.0272,17.587100,33.635860
1252,19950503,520.479980,520.539978,514.859985,1995-05-03,456.137767,400.120848,True,1995-05-03,1993-03-10,784.0,0.1169,0.0616,1995,0.0230,16.311082,49.946943
1385,19951109,593.260010,593.900024,590.890015,1995-11-09,519.997054,456.137767,True,1995-11-09,1995-05-03,190.0,0.1246,0.0543,1995,0.0230,20.585082,70.532024
1519,19960522,678.419983,678.419983,671.229980,1996-05-22,592.796642,519.997054,True,1996-05-22,1995-11-09,195.0,0.1550,0.0556,1996,0.0201,31.563208,102.095232
1682,19970114,768.859985,772.039978,759.510010,1997-01-14,675.788171,592.796642,True,1997-01-14,1996-05-22,237.0,0.1927,0.0560,1997,0.0160,46.794672,148.889904
1786,19970612,883.460022,884.340027,869.010010,1997-06-12,770.398515,675.788171,True,1997-06-12,1997-01-14,149.0,0.1851,0.0568,1997,0.0160,52.149998,201.039902
1947,19980202,1001.270020,1002.479980,980.280029,1998-02-02,878.254308,770.398515,True,1998-02-02,1997-06-12,235.0,0.2136,0.0526,1998,0.0132,69.340240,270.380142
2047,19980625,1129.280029,1142.040039,1127.599976,1998-06-25,1001.209911,878.254308,True,1998-06-25,1998-02-02,143.0,0.2025,0.0541,1998,0.0132,76.521416,346.901558
2225,19990311,1297.680054,1306.430054,1286.839966,1999-03-11,1141.379298,1001.209911,True,1999-03-11,1998-06-25,259.0,0.2437,0.0477,1999,0.0114,104.421582,451.323140


#### Step 08: list all put purchases when rolling to a lower strike.
In this case, you are selling put spreads.

In [11]:
dft[(dft.time_to_hedge) & (dft.prev_hedge_strike>dft.current_hedge_strike)]

,settle_date,close,high,low,settle_dt,current_hedge_strike,prev_hedge_strike,time_to_hedge,hedge_date,prev_hedge_date,days_of_hedge,atm_vol,rate,year,div_yield,hedge,hedge_cumulative
2861,20010921,965.799988,984.539978,944.750000,2001-09-21,1119.008264,1301.172400,True,2001-09-21,2000-03-21,549.0,0.4266,0.0253,2001,0.0137,-120.778868,424.523777
3069,20020722,819.849976,854.130005,813.260010,2002-07-22,962.347107,1119.008264,True,2002-07-22,2001-09-21,304.0,0.4187,0.0194,2002,0.0181,-105.292645,319.231132
4636,20081009,909.919983,1005.250000,909.190002,2008-10-09,1075.573018,1250.666300,True,2008-10-09,2006-12-14,665.0,0.6392,0.0133,2008,0.0315,-118.667869,271.752338
4666,20081120,752.440002,820.520020,747.780029,2008-11-20,924.992796,1075.573018,True,2008-11-20,2008-10-09,42.0,0.8086,0.0087,2008,0.0315,-106.030046,165.722292
4736,20090305,682.549988,708.270020,677.929993,2009-03-05,795.493804,924.992796,True,2009-03-05,2008-11-20,105.0,0.5017,0.0066,2009,0.0197,-88.316020,77.406272


#### Step 09: plot the dramatic roll down cases during 2008-2010

In [12]:
dft_to_plot = dft[['settle_date','close','current_hedge_strike']]
dft_to_plot = dft_to_plot[(dft_to_plot.settle_date>=20080501)]

iplot(plotly_plot(df_in=dft_to_plot,x_column='settle_date'))

#### Step 10: show comparative PL's

In [13]:
row_min = dft[dft.settle_dt == dft.settle_dt.min()].iloc[0]
row_max = dft[dft.settle_dt == dft.settle_dt.max()].iloc[0]
years_of_position = (row_max.settle_dt - row_min.settle_dt).days/365
beg_value = row_min.close
curr_value  = row_max.close
curr_return  = (curr_value/beg_value - 1)**(1/years_of_position) - 1

highest_high_value = dft[dft.high==dft.high.max()].iloc[0].close
highest_return_no_hedge = (highest_high_value/beg_value - 1)**(1/years_of_position) - 1

hedge_cost = dft[dft.time_to_hedge].hedge.sum()
hedged_value = row_max.current_hedge_strike - hedge_cost
hedged_return = (hedged_value/beg_value - 1)**(1/years_of_position) - 1



#### Step 11: Calculate returns from a portfolio that holds a ratio of stock and 1 year treasury bonds
(*When you rebalance the portfolio after it achieves a certain threshold*)


In [14]:
# get the initial shares of stock and cash
shares = rebal_target / dft.close[0]
cash = 1 - rebal_target
# set up arrays to accumlate daily changes
cash_per_day = []
stock_per_day = []
port_per_day = []
prices = dft.close.values
dates = dft.settle_date.values
cash_rates = dft.rate.values / 365
rebal_dates = []
rebal_sales = []
stock_percs = []

# main loop to determine portfolio values over time, and to determine when to rebalance
for i in range(1,len(dft)):
    # calculate current stock dollars
    stock_dollars = shares * prices[i]
    # have your cash earn interest each day
    cash_rate = cash_rates[i]
    cash = cash * (1+cash_rate)
    # determine portfolio value 
    port = stock_dollars + cash
    # determine pre-rebalance stock percent
    stock_perc = stock_dollars/port
    stock_percs.append(stock_perc)
    # determine if you should rebalance
    if stock_perc >= rebal_adjust:
        # do re-balance
        dollars_to_sell = stock_dollars - rebal_target*port
        new_stock_dollars = stock_dollars - dollars_to_sell
        new_cash = cash + dollars_to_sell
        new_port = new_stock_dollars + new_cash
        shares = new_stock_dollars/prices[i]
        cash = new_cash
        stock_dollars = new_stock_dollars
        rebal_dates.append(dates[i])
        rebal_sales.append(dollars_to_sell)
    cash_per_day.append(cash)
    stock_per_day.append(stock_dollars)
    port_per_day.append(cash+stock_dollars)    



### Display results of portfolio analysis above, comparing:
1. Annualized Returns from 100% long
2. Highest Historical Return from 100% long
3. Annualized Return of put strategy portfolio
4. Annualized Return of rebalanced portfolio

In [15]:
# get total years and calculate annualized portfolio performance
total_days = (dft.settle_dt.values[-1] - dft.settle_dt.values[0]).astype('timedelta64[D]')// np.timedelta64(1, 'D')
total_years = total_days / 365
end_port_value = port_per_day[-1]
beg_port_value = port_per_day[0]
annualized_port_yield = round((end_port_value/beg_port_value)**(1/total_years) - 1,3)
df_values = pd.DataFrame({
    'return_type':['total years','annualized current return',
                   f'annualized highest return',f'annualized current hedged return {round(put_perc_otm*100,1)}%',
                  f'rebalanced ({int(rebal_target*100)}%,{int(rebal_adjust*100)}%) portfolio end value'],
    'current_value':[total_years,curr_value,highest_high_value,hedged_value,end_port_value],
    'return':[0,curr_return,highest_return_no_hedge,hedged_return,annualized_port_yield]})
display.display(HTML("<h4 style='text-align:center;'>Compare Strategy Results<hr>" + df_values.style.render()))

,return_type,current_value,return
0,total years,29.967123,0.000000
1,annualized current return,2830.709961,0.066613
2,annualized highest return,3386.149902,0.073850
3,annualized current hedged return 14.0%,1991.464932,0.051958
4,"rebalanced (60%,70%) portfolio end value",4.726181,0.053000


In [16]:
df_stock_perc = pd.DataFrame(
    {'dt':dft.settle_date.values[1:],'stock_perc':stock_percs,
#     'close':dft.close.values[1:]})
    'port':port_per_day})


title = f"""Stock vs Cash changes over time<br>
along the value of 1 Portfolio dollar over Time.<br>
(Net portfolio change = {annualized_port_yield})
"""

iplot(plotly_plot(
    df_in=df_stock_perc,x_column='dt',yaxis2_cols=['port'],
    plot_title=title,
    y_left_label='Percent of Portfolio in Stock',
    y_right_label='Value of Portfolio'))

                  

### Create a dataframe that holds a "base" volskew, representing a volskew with:
1. the median S&P vol skew 10% below the money, for options on the CME ES contract from 2011 to 2020
2. the vol skew of the rows from number 1, that has the median skew 10% above the money

In [17]:
df_iv_skew_ES = pd.read_csv('iv_skew_ES_2011_2020.csv')

df_iv_skew_ES2 = df_iv_skew_ES[df_iv_skew_ES['-0.1'] == df_iv_skew_ES['-0.1'].median()]
df_iv_skew_ES2 = df_iv_skew_ES2[df_iv_skew_ES2['0.1'] == df_iv_skew_ES2['0.1'].median()]
df_only_strikes = df_iv_skew_ES2.iloc[0:1][[c for c in df_iv_skew_ES2.columns.values if '.' in c]]
strikes = [round(float(c),2) for c in  df_only_strikes.columns.values]
skews = df_only_strikes.iloc[0].values
df_skew = pd.DataFrame({'strike':strikes,'vol_skew':skews})
df_skew.index = df_skew.strike
df_skew = df_skew[['vol_skew']]
def _calc_skew_from_df_skew(underlying_price,strike):
    curr_perc_otm = round(strike/underlying_price - 1,5)
    df_skew2 = df_skew.copy()
    row_to_append = df_skew2.iloc[-1]
    row_to_append.name = curr_perc_otm
    row_to_append['vol_skew'] = np.nan
    df_skew2 = df_skew2.append(row_to_append)
    df_skew2 = df_skew2.sort_index()
    df_skew2['vol_skew'] = df_skew2.vol_skew.interpolate(method='polynomial', order=2)
    skew = df_skew2.loc[curr_perc_otm,'vol_skew']
    return skew




In [18]:
# test it - value should be about 0.0653434
_calc_skew_from_df_skew(2900,2900*(1-.12345))


0.065343393830266

In [19]:
def _same_strike_hedge(r):  
    skew_for_hedge = _calc_skew_from_df_skew(r.close,r.current_hedge_strike)
    hedge = black.black('p', r.close, r.current_hedge_strike, years_to_hedge, .02, skew_for_hedge)
    return hedge
    
dft_hedge_noroll = dft[(dft.days_of_hedge==years_to_hedge*365) & (dft.time_to_hedge==False)]
dft_hedge_noroll.apply(_same_strike_hedge,axis=1).sum(),dft.hedge.sum()

(4.807831568677079, 957.6066548030537)

In [20]:
for _ in range(3):
    os.system("echo -ne '\007'")


## END